# Reto 5: IA Responsable

A medida que los LLMs crecen en popularidad y uso en todo el mundo, la necesidad de gestionar y monitorear sus salidas se vuelve cada vez más importante. En este desafío, aprenderás cómo evaluar las salidas de los LLMs y cómo identificar y mitigar posibles sesgos en el modelo.

Preguntas que deberías poder responder al final de este desafío:
- ¿Cómo puedes aprovechar el filtrado de contenido?
- ¿Cuáles son las formas de evaluar la veracidad y reducir las alucinaciones?
- ¿Cómo puedes identificar y mitigar el sesgo en tu modelo?

Secciones en este Desafío:

1. Identificar daños y detectar Información Personal Identificable (PII)<!--(#content-filtering,-content-safety,-and-personal-identifiable-information-(pii)-detection)-->
1. Evaluar la veracidad utilizando Conjuntos de Datos de Verdad Fundamental (Ground Truth Datasets)<!--(#evaluating-truthfulness-using-ground-truth-data)-->
1. Evaluar la veracidad utilizando GPT sin Conjuntos de Datos de Verdad Fundamental<!--(#evaluating-truthfulness-using-gpt-without-ground-truth-datasets)-->

Recursos:
- [Descripción general de las prácticas de IA Responsable para modelos de Azure OpenAI](https://learn.microsoft.com/en-us/legal/cognitive-services/openai/overview)

## 1. Filtrado de Contenido, Content Safety y Detección de Información Personal Identificable (PII)

Las cuatro etapas de las recomendaciones de IA Responsable al usar OpenAI son identificar, medir, mitigar y operar daños. En esta sección, nos centraremos en identificar daños.

Este paso tiene como objetivo identificar posibles daños para que puedas mitigarlos eficazmente. Es importante recordar que identificar daños depende en gran medida del contexto. Por ejemplo, un modelo que se utiliza para generar texto para un libro infantil tendrá diferentes daños que un modelo que se utiliza para generar texto para un artículo de noticias. El lenguaje también tendrá diferentes significados en diferentes contextos, por lo que un marco de identificación debe ser lo suficientemente flexible como para adaptarse a varias situaciones.

Presentamos tres herramientas para identificar daños:
- Azure AI Services Content Filtering (Filtrado de Contenido)
- Azure AI Content safety
- Detección de PII a través de Plugins de OpenAI

### 1.1 Azure AI Services Content Filtering

De acuerdo a la [documentación de Azure](https://learn.microsoft.com/es-mx/azure/cognitive-services/openai/concepts/content-filter): 

    El Servicio Azure OpenAI incluye un sistema de gestión de contenido que trabaja junto a los modelos principales para filtrar contenido. Este sistema funciona ejecutando tanto el prompt de entrada como el contenido generado a través de un conjunto de modelos de clasificación destinados a detectar el uso indebido.

Debes evaluar cuidadosamente todos los posibles daños y agregar mitigaciones específicas del escenario según sea necesario. Por ejemplo, es posible que desees filtrar contenido que sea ofensivo, profano, sexualmente explícito o lleno de odio.

**Verificación de Conocimientos #1:**:

Para evaluar tu comprensión del concepto de filtrado de contenido, responde las siguientes preguntas basándote en la documentación:

* Verdadero o Falso: Si realizas una solicitud de completado en streaming para múltiples respuestas, el filtro de contenido evaluará cada respuesta individualmente y devolverá solo las que pasen.
* Verdadero o Falso: el parámetro `finish_reason` se devolverá en cada respuesta del filtro de contenido.
* Verdadero o Falso: Si el sistema de filtrado de contenido no está disponible, no podrás recibir resultados sobre tu solicitud.

### 1.2 Azure AI Content Safety

[Azure AI Content Safety](https://learn.microsoft.com/es-mx/azure/cognitive-services/content-safety/overview) fue creado para ayudar a las organizaciones a gestionar y moderar de manera responsable el contenido generado por usuarios y por la IA. Es un servicio administrado que proporciona una solución de moderación de contenido escalable, de baja latencia y rentable para tu contenido de imágenes y texto. Está diseñado para ayudarte a detectar contenido potencialmente inseguro, incluyendo discurso de odio, violencia, material sexualmente explícito y autolesiones, entre otras capacidades.

Puedes leer más sobre el servicio en este [artículo de Microsoft](https://techcommunity.microsoft.com/t5/ai-cognitive-services-blog/introducing-azure-ai-content-safety-helping-organizations-to/ba-p/3825744).

**Verificación de Conocimientos #2**:

Verifica tu comprensión del Servicio AI Content Safety respondiendo las siguientes preguntas:

* Verdadero o Falso: La API de Moderación de Texto está diseñada para admitir más de 100 idiomas como entrada.
* Verdadero o Falso: El Servicio AI Content Safety tiene una función para monitorear estadísticas de actividad de tu aplicación.
* Verdadero o Falso: Azure AI Content Safety Studio y la API tienen diferentes puntuaciones de riesgo (niveles de severidad) en las categorías de daño.
* Verdadero o Falso: Solo puedes personalizar los umbrales de severidad a través de la API.
* Verdadero o Falso: La API siempre devuelve un nivel de severidad para las cuatro categorías de contenido.

Para ejecutar el ejemplo, primero instala algunos paquetes y carga tus variables de entorno desde un archivo `.env`.

**NOTA:** El soporte de la biblioteca openai-python para Azure OpenAI está en vista previa. Hemos especificado la versión de API Preview a continuación.

`os.getenv()` asume que estás usando variables de entorno para el endpoint y la clave.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
CHAT_MODEL = os.getenv("CHAT_MODEL_NAME")
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.environ['OPENAI_API_TYPE']
CHAT_INSTRUCT_MODEL = os.getenv("CHAT_MODEL_NAME")
openai.api_version = "2023-06-01-preview" # API version required to test out Annotations preview

A continuación se muestra un ejemplo de llamada a OpenAI utilizando la versión Preview que habilita las Anotaciones ([Annotations](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/content-filter?tabs=warning%2Cuser-prompt%2Cpython-new#annotations)). Reemplaza el prompt de entrada con diferentes textos para ver cómo cambian las anotaciones.

In [ ]:
pii_prompt = "{Example prompt where a severity level of low is detected}"

In [ ]:
response = openai.Completion.create(
    engine=CHAT_MODEL,
    prompt=pii_prompt 
    # Content that is detected at severity level medium or high is filtered, 
    # while content detected at severity level low isn't filtered by the content filters.
)
print(response)

### 1.3 Verificación de datos PII

Los plugins son extensiones de chat diseñadas específicamente para modelos de lenguaje como ChatGPT, lo que les permite acceder a información actualizada, ejecutar cálculos o interactuar con servicios de terceros en respuesta a la solicitud de un usuario. Desbloquean una amplia gama de casos de uso potenciales y mejoran las capacidades de los modelos de lenguaje.

La función mostrada a continuación, `screen_text_for_pii`, puede ser útil si deseas evitar subir documentos sensibles o privados a una base de datos de manera involuntaria.

Esta característica no es infalible y puede no detectar todos los casos de información personal identificable. Utiliza esta característica con precaución y verifica su efectividad para tu caso de uso específico. Puedes leer más sobre el contexto de esta función en OpenAI [aquí](https://github.com/openai/chatgpt-retrieval-plugin/tree/main#plugins).

Para otras formas de garantizar que tus datos estén seguros cuando uses OpenAI, consulta las formas de [configurar el servicio OpenAI con identidades gestionadas](https://learn.microsoft.com/es-mx/azure/cognitive-services/openai/how-to/managed-identity).

Lee la función `screen_text_for_pii` en la celda siguiente para entender cómo funciona. Puedes reemplazar el texto de entrada con información relevante para tu caso de uso.

In [ ]:
def get_completion_from_messages(messages, model=CHAT_MODEL, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def screen_text_for_pii(text: str) -> bool:
    # This prompt is just an example, change it to fit your use case
    messages = [
        {
            "role": "system",
            "content": f"""
            You can only respond with the word "True" or "False", where your answer indicates whether the text in the user's message contains PII.
            Do not explain your answer, and do not use punctuation.
            Your task is to identify whether the text extracted from your company files
            contains sensitive PII information that should not be shared with the broader company. Here are some things to look out for:
            - An email address that identifies a specific person in either the local-part or the domain
            - The postal address of a private residence (must include at least a street name)
            - The postal address of a public place (must include either a street name or business name)
            - Notes about hiring decisions with mentioned names of candidates. The user will send a document for you to analyze.
            """,
        },
        {"role": "user", "content": text},
    ]

    completion = get_completion_from_messages(messages)
    
    if completion.startswith("True"):
        return True

    return False

In [ ]:
# Optional: test out the screening for PII using input data
text = "INPUT YOUR TEXT HERE"
screen_text_for_pii(text)

## 2. Evaluar la veracidad utilizando datos de Ground Truth (Verdad Fundamental)

En esta sección, nos centraremos en evaluar la veracidad en las salidas del modelo. Las alucinaciones del modelo son un problema común al usar LLMs, por lo que es importante evaluar si el modelo está generando respuestas basadas en datos en lugar de inventar información. El objetivo es mejorar la veracidad de los resultados para que tu modelo sea más consistente y fiable para la producción.

Esta sección se centrará en cómo evaluar tu modelo cuando tienes acceso a datos de [Ground Truth](https://es.wikipedia.org/wiki/Verdad_fundamental). Esto nos permitirá comparar la salida del modelo con la respuesta correcta. En la siguiente sección, nos centraremos en cómo evaluar tu modelo cuando no tienes acceso a datos de Verdad Fundamental.

Cuando usamos datos de Verdad Fundamental, podemos deducir una representación numérica de cuán similar es la respuesta generada a la respuesta correcta utilizando varias métricas. También tendrás la oportunidad de identificar e implementar métricas adicionales para evaluar el caso de uso en esta sección.

Evaluaremos modelos utilizando conjuntos de datos de Hugging Face, así como la biblioteca [Evaluate de Hugging Face](https://huggingface.co/docs/evaluate/index).

También utilizaremos LangChain, que tiene un paquete (QAEvalChain) para este propósito específico. [Lee más](https://python.langchain.com.cn/docs/guides/evaluation/question_answering) sobre cómo LangChain implementa Evaluation. Puede que hayas oído hablar de LangChain y Semantic Kernel. LangChain es un framework de código abierto de terceros que puedes usar para desarrollar aplicaciones impulsadas por modelos de lenguaje. LangChain facilita las complejidades de trabajar y construir con modelos de IA proporcionando el marco de orquestación de pipelines y utilidades de ayuda para ejecutar poderosos pipelines de múltiples modelos. También se puede integrar con Prompt Flow para escalar los flujos de trabajo de ingeniería de prompts.

Al final de esta sección, podrás revisar qué enfoque (Evaluate de Hugging Face o QAEvalChain de LangChain) es preferible para futuros casos de uso.

### 2.1 Configuración

Para fines de demostración, evaluaremos un sistema simple de respuesta a preguntas.

In [ ]:
! pip install langchain-community langchain-core

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI

Ahora crearemos un Prompt Template (Plantilla de Prompt) que nos permitirá usar el mismo prompt con diferentes entradas. Utilizaremos [LangChain](https://docs.langchain.com/docs/), un framework de código abierto para trabajar con modelos de lenguaje.

Lee más sobre LangChain Chains y cómo funcionan [aquí](https://docs.langchain.com/docs/components/chains/).

In [ ]:
prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])
llm = AzureChatOpenAI(deployment_name=CHAT_MODEL, temperature=0.9)
chain = LLMChain(llm=llm, prompt=prompt)

### 2.2 Carga de datos

Ahora cargamos un conjunto de datos de Hugging Face y luego lo convertimos en una lista de diccionarios para un uso más fácil.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("truthful_qa", "generation")

Trabajemos con los primeros cinco ejemplos en el [conjunto de datos Truthful QA de Hugging Face](https://huggingface.co/datasets/truthful_qa). Estamos trabajando con la subsección "Generation" del conjunto de datos porque lo estamos aplicando a un sistema de generación de texto, pero observa que hay otra subsección para evaluar el rendimiento del modelo en escenarios de opción múltiple.

In [ ]:
num_examples = 3
examples = list(dataset['validation'])[:num_examples]

In [ ]:
examples[0]

### 2.3 Predicciones

Ahora podemos hacer y examinar las predicciones para estas preguntas.

In [ ]:
predictions = chain.apply(examples)

In [ ]:
predictions

### 2.4 Evaluación
Podemos ver que si intentáramos hacer una coincidencia exacta en las respuestas, no coincidirían con lo que respondió el modelo de lenguaje. Sin embargo, semánticamente el modelo de lenguaje es correcto en ambos casos. Para tener esto en cuenta, podemos usar un modelo de lenguaje para evaluar las respuestas.

Debido a que estas respuestas son más complejas que las de opción múltiple, ahora podemos evaluar su precisión utilizando un modelo de lenguaje.

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
# Create an Evaluation Chain using LangChain's QAEValChain
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(examples, predictions, question_key="question", answer_key="best_answer", prediction_key="text")

In [ ]:
graded_outputs

Ahora vamos a contar el número de salidas que fueron calificadas como "Correct" (Correctas) o "Incorrect" (Incorrectas) según la evaluación de QAEvalChain.

In [ ]:
num_correct = sum([1 for x in graded_outputs if str(x['results']).upper().startswith('CORRECT')])
num_incorrect = sum([1 for x in graded_outputs if str(x['results']).upper().startswith('INCORRECT')])

In [ ]:
print(num_correct, num_incorrect)

### 2.5 Comparación con otras métricas de evaluación

Podemos comparar los resultados de la evaluación que obtenemos con otras métricas de evaluación comunes. Para hacer esto, carguemos algunas métricas de evaluación del paquete Evaluate de HuggingFace.

In [ ]:
print(examples[0])

In [ ]:
# Some data munging to get the examples in the right format
for i, eg in enumerate(examples):
    eg['id'] = str(i)
    eg['answers'] = {"text": eg['correct_answers'], "answer_start": [0]}
    predictions[i]['id'] = str(i)
    predictions[i]['prediction_text'] = predictions[i]['text']

for p in predictions:
    del p['text']

# references need id, answers as list with text and answer_start
new_examples = examples.copy()
# print(new_examples)
for eg in new_examples:
    del eg ['question']
    del eg['best_answer']
    del eg['type']
    del eg['correct_answers']
    del eg['category']
    del eg['incorrect_answers']
    del eg['source']

In [ ]:
from evaluate import load
squad_metric = load("squad")
results = squad_metric.compute(
    references=new_examples,
    predictions=predictions,
)

In [ ]:
results

#### (Opcional) Tarea del Estudiante

Ahora agrega dos métricas adicionales para evaluar el modelo utilizando la biblioteca Evaluate de Hugging Face. Una de esas métricas podría ser la métrica BERT_score.

Recursos de referencia:

* [Biblioteca Evaluate de Hugging Face en GitHub](https://github.com/huggingface/evaluate) 
* [Documentación de la Biblioteca Evaluate](https://huggingface.co/docs/transformers/tasks/translation#evaluate) 

In [ ]:
### TAREA DEL ESTUDIANTE ###

## 3. Evaluación de Modelos para determinar su Veracidad mediante GPT sin Conjuntos de Datos de Verdad Fundamental

No siempre tendrás datos de Ground Truth (Verdad Fundamental) disponibles para evaluar tu modelo. Afortunadamente, GPT hace un muy buen trabajo al generar datos de Verdad Fundamental a partir de tu conjunto de datos original.

La investigación ha demostrado que los LLMs como GPT-3 y ChatGPT son buenos para evaluar la inconsistencia del texto. Basándonos en estos hallazgos, los modelos pueden usarse para evaluar la veracidad de las oraciones mediante prompts a GPT. Evaluemos la precisión de GPT a través de una técnica en la que GPT se evalúa a sí mismo.

En esta sección, evaluaremos el modelo en el que trabajaste en el desafío anterior aplicado al conjunto de datos CNN Dailymail.

In [ ]:
from langchain.chains import LLMChain, QAGenerationChain
from langchain.requests import Requests
from langchain.llms import AzureOpenAI
from langchain.document_loaders import TextLoader
import pandas as pd
import json

### 3.1. Crear un Conjunto de Datos de Verdad Fundamental en Datos Personalizados
Comencemos utilizando GPT para crear un conjunto de datos de pares de preguntas y respuestas como nuestros datos de "verdad fundamental" a partir del conjunto de datos CNN Dailymail del desafío anterior.

In [ ]:
# Load the provided CNN file, the path of which may change based on folder structure
CNN_FILE_PATH = "../data/structured/cnn_dailymail_data.csv"

# Optional: limit to 11 samples for simple scope to avoid RateLimitErrors
# You are welcome to change `num_samples` or delete it to run this example on
# the entire dataset but doing so may take 1+ hour
num_samples = 11
df = pd.read_csv(CNN_FILE_PATH)[:num_samples]
df.drop([4,9], axis=0, inplace=True)
df = df.drop(columns=["highlights"])
pd.set_option('display.max_colwidth', None)  # Show all columns

In [ ]:
# Take a look at the data
df.head(3)

Es hora de hacer un poco de limpieza de datos para garantizar la consistencia.

In [ ]:
# Convert the column "article" to a list of dictionaries
df_copy = df.copy().rename(columns={"article": "text"})
df_copy = df_copy.drop(columns=["id"])
df_dict = df_copy.to_dict("records")

print(df_dict)

Hemos generado un par de preguntas y respuestas para cada artículo. Esto nos ayudará a evaluar el rendimiento de GPT en cuanto a la calidad de sus respuestas a las preguntas de prueba. Las respuestas en cada par se consideran nuestros datos de verdad fundamental y la respuesta ideal.

Creamos estos pares utilizando [QAGenerationChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.qa_generation.base.QAGenerationChain.html#) de Langchain. Consulta el [código fuente](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/qa_generation) para ver cómo se generan los pares de preguntas y respuestas a través de QAGenerationChain. ¡La implementación puede sorprenderte!

En el proceso, eliminamos los artículos que activaron el filtro de contenido de OpenAI.

A continuación, vamos a cargar el conjunto de datos de preguntas y respuestas proporcionado para la evaluación posterior.

In [ ]:
llm = AzureOpenAI(deployment_name=CHAT_MODEL, temperature=0, max_tokens=1000)
chain = QAGenerationChain.from_llm(llm)

In [ ]:
# Load cnn_qa_set.json
cnn_qa_set_filepath = '../data/structured/cnn_qa_set.json'
with open(cnn_qa_set_filepath, 'r') as file:
    qa_set = json.load(file)

In [ ]:
qa_set[:3]

Ahora tenemos las preguntas y las respuestas de Verdad Fundamental. ¡Probemos la solución de GPT + AI Search que implementaste en el último desafío! Vamos a comparar las diferencias entre `truth_answers` (respuestas proporcionadas) y `prompt_answers` (desempeño del modelo).

In [ ]:
questions = [(set["question"] for set in qa_set)]
truth_answers = [(set["answers"] for set in qa_set)]
prompt_answers = list()

### 3.2 Instanciar el Índice de AI Search

Estamos usando el Índice que creaste en el último desafío para recuperar documentos que sean relevantes para cualquier consulta de usuario de entrada.

In [ ]:
import os, json, requests, sys, re
import requests
from pprint import pprint
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings
)

import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity

In [ ]:
# Create an SDK client
service_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")   
key = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")
credential = AzureKeyCredential(key)
index_name = os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME")

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [ ]:
# Create a pandas dataframe with columns from qa_set
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(qa_set)
df = df.rename(columns={"answer": "truth_answer"})
df.head(3)

Vamos a recuperar los artículos relevantes para cada pregunta en nuestro dataframe `qa_set`.

In [ ]:
# Get the articles for the search terms
# Optional: change `num_docs` to change how many relevant ranked documents the Search index should return
num_docs=1
for i, row in df.iterrows():
    search_term = row['question']
    results = search_client.search(search_text=search_term, include_total_count=num_docs)
    df.loc[i, "context"] = next(results)['article']
df.head(3)

Usando una plantilla de prompt (prompt template), podemos introducir preguntas a GPT utilizando la información de los documentos recuperados.

Observa qué modelo estamos usando ahora para generar respuestas. ¿A qué se debe esto? ¿Qué sucede si utilizas el modelo de chat que usamos anteriormente?

In [ ]:
from langchain.prompts import PromptTemplate

# Ask the model using the embeddings from Challenges 3 and 4 to answer the questions
template = """You are a search assistant trying to answer the following question. Use only the context given. Your answer should only be one sentence.

    > Question: {question}
    
    > Context: {context}"""

# Create a prompt template
prompt = PromptTemplate(template=template, input_variables=["question", "context"])
llm = AzureOpenAI(deployment_name=CHAT_INSTRUCT_MODEL, temperature=0)
search_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

prompt_answers = []
for question, context in list(zip(df.question, df.context)):
    response = search_chain.run(question=question, context=context)
    prompt_answers.append(response.replace('\n',''))
df['prompt_answer'] = prompt_answers   

Examina las tres primeras respuestas del modelo basadas en los artículos. ¿Cómo podrías utilizar técnicas de Prompt Engineering para refinar las respuestas?

In [ ]:
df['prompt_answer'].head(3)

Después de generar respuestas a nuestras preguntas de prueba, podemos usar GPT (puede ser otro modelo si lo prefieres, como GPT-4) para evaluar la exactitud de nuestras respuestas de Verdad Fundamental utilizando una rúbrica.

Observa cómo el prompt está utilizando técnicas que aprendiste en los Desafíos 1 y 2.

In [ ]:
eval_template = """You are trying to answer the following question from the context provided:

> Question: {question}

The correct answer is:

> Query: {truth_answer}

Is the following predicted query semantically the same (eg likely to produce the same answer)?

> Predicted Query: {prompt_answer}

Please give the Predicted Query a grade of either an A, B, C, D, or F, along with an explanation of why. End the evaluation with 'Final Grade: <the letter>'

> Explanation: Let's think step by step."""

In [ ]:
eval_prompt = PromptTemplate(template=eval_template, input_variables=["question", "truth_answer", "prompt_answer"])

In [ ]:
# Create a new LLM Chain to submit the prompt we created
eval_chain = LLMChain(llm=llm, prompt=eval_prompt, verbose=False)

# Submit the prompt using our dataset
eval_results = []
for question, truth_answer, prompt_answer in list(zip(df.question, df.truth_answer, df.prompt_answer)):
    eval_output = eval_chain.run(
        question=question,
        truth_answer=truth_answer,
        prompt_answer=prompt_answer,
    )
    eval_results.append(eval_output)
eval_results

Ahora vamos a analizar los resultados de la rúbrica para cuantificarlos y resumirlos en conjunto.

In [ ]:
import re
from typing import List
from collections import defaultdict

# Parse the evaluation chain responses into a rubric
def parse_eval_results(results: List[str]) -> List[float]:
    rubric = {
        "A": 1.0,
        "B": 0.75,
        "C": 0.5,
        "D": 0.25,
        "F": 0
    }
    return [rubric[re.search(r'Final Grade: (\w+)', res).group(1)] for res in results]

scores = defaultdict(list)
parsed_results = parse_eval_results(eval_results)

# Collect the scores for a final evaluation table
scores['request_synthesizer'].extend(parsed_results)

In [ ]:
# Reusing the rubric from above, parse the evaluation chain responses
parsed_eval_results = parse_eval_results(eval_results)
# Collect the scores for a final evaluation table
scores['result_synthesizer'].extend(parsed_eval_results)

# Print out Score statistics for the evaluation session
header = "{:<20}\t{:<10}\t{:<10}\t{:<10}".format("Metric", "Min", "Mean", "Max")
print(header)
for metric, metric_scores in scores.items():
    mean_scores = sum(metric_scores) / len(metric_scores) if len(metric_scores) > 0 else float('nan')
    row = "{:<20}\t{:<10.2f}\t{:<10.2f}\t{:<10.2f}".format(metric, min(metric_scores), mean_scores, max(metric_scores))
    print(row)

¡Ahí lo tienes! Ahora podemos revisar los resultados de la evaluación del modelo en conjunto con Azure AI Search del último desafío. Puedes realizar un análisis similar en tu caso de uso y datos personalizados.

## Conclusión

En este desafío, cubrimos los principios de la IA Responsable, particularmente al trabajar con OpenAI, y cómo evaluar el rendimiento de una implementación de modelo utilizando datos de Verdad Fundamental.

Te presentamos varias herramientas y servicios, algunos de Azure y otros que son de código abierto. Puedes referirte a ellos para tus propios proyectos y decidir cuál funciona mejor para tus escenarios.

**Respuestas a la Verificación de Conocimientos #1:**:
* Verdadero
* Falso - se devolverá si no se considera inapropiado
* Falso - tu solicitud aún se completará sin filtrado de contenido. Puedes ver si no se aplicó buscando un mensaje de error en el objeto `content_filter_result`.

**Respuestas a la Verificación de Conocimientos #2:**:

* Falso: El servicio se entrenó en más de 100 idiomas, pero está diseñado para admitir solo unos pocos.
* Verdadero: Content Safety tiene una página de monitoreo para ayudarte a rastrear el rendimiento y las tendencias de tu API de moderación e informar tu estrategia de moderación de contenido.
* Verdadero: El Studio utiliza cuatro niveles de riesgo, mientras que la API puntúa el riesgo en una escala del 0 al 6.
* Falso: También puedes personalizar los umbrales de severidad en el Studio.
* Falso: Puedes especificar en qué categorías deseas evaluar tu texto en la API utilizando el parámetro `categories`.